# Reinforcement Learning in Mario - Tutorial

This tutorial will gude you through the setup process for the Mario Environment and Stablebaselines library

## First steps

- Create a conda virtual environment with Python 3.9
    - Make sure you have anaconda installed in your computer - (Conda Installation)[https://conda.io/projects/conda/en/latest/user-guide/install/index.html]
    -  Open the Conda console and use the following line to create the virtual environment: 
        ```console
        user@pc:~$ conda create --name my_environment python=3.9
        user@pc:~$ conda activate my_environment
        ```
        *Note the new environment only has Python, we will install the dependencies along the way to detect potential compatibility issues.

- Setup the VScode IDE to run Python from the new environment
    - Open the command palette, use the following shortcut: Control + Shift + P or under the menu "view."
    - Type in the textbox "select interpreter" and choose the python from the new virtual environment.
    - For further information about this process, follow the link: (Python environments in VS Code)[https://code.visualstudio.com/docs/python/environments]